# NDEx2 Tutorial

In this tutorial you will learn to use NiceCX, a simple data model that is part of the ndex2 NDEx Client module.
NiceCX facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires Python 3.6+ and the ndex2 module, see the NDEx Client Tutorial for installation instructions.

### Import Packages Required for this Tutorial

In [2]:
from nicecxModel.NiceCXNetwork import NiceCXNetwork
from nicecxModel.cx.aspects.NodesElement import NodesElement
from nicecxModel.cx.aspects.EdgesElement import EdgesElement
from nicecxModel.cx.aspects.NodeAttributesElement import NodeAttributesElement
from nicecxModel.cx.aspects.EdgeAttributesElement import EdgeAttributesElement
import networkx as nx
import pandas as pd

# Five Ways to Work with NiceCX networks:

- Starting with an Empty Network
- CX Files
- NDEx Networks
- NetworkX Networks
- Pandas DataFrames

## Starting with an Empty Network

**Create an empty niceCx network**

In [3]:

niceCx_creatures = NiceCXNetwork()

**Populate the Network**

The _niceCx_creatures_ will now be populated with data in which each node represents a species and has a color attribute.Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [4]:
niceCx_creatures.setName("Food Web")

**Add Nodes and Edges**

In [5]:
fox_node = niceCx_creatures.addNode(node_name='Fox')
mouse_node = niceCx_creatures.addNode(node_name='Mouse')
bird_node = niceCx_creatures.addNode(node_name='Bird')

fox_bird_edge = niceCx_creatures.addEdge(edge_source=fox_node, edge_target=bird_node, edge_interaction='interacts-with')

fox_mouse_edge = niceCx_creatures.addEdge(edge_source=fox_node, edge_target=mouse_node, edge_interaction='interacts-with')


The addNode and addEdge methods return the unique ID assigned to the new node or edge. In CX, IDs are always assigned in an ascending order, although they may not always be sequential In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

**Add Attributes**

The addNodeAttribute and addEdgeAttribute require the ID in the "property_of" field and the property "name" and "values" in their respective fields.

In [6]:
niceCx_creatures.addNodeAttribute(property_of=fox_node, name='Color', values='Red')

niceCx_creatures.addNodeAttribute(property_of=mouse_node, name='Color', values='Gray')

niceCx_creatures.addNodeAttribute(property_of=bird_node, name='Color', values='Blue')

In [7]:
niceCx_creatures.addEdgeAttribute(property_of=fox_mouse_edge, name='Hunted', values='On the ground')

We can now print a summary of niceCX_creatures. The returned value of the getSummary() method is structured in the same format as NetworkSummary objects returned by NDEx network search methods.

In [8]:
print(niceCx_creatures.getSummary())

AttributeError: 'NiceCXNetwork' object has no attribute 'getSummary'

## CX Files

In [ ]:
niceCx = NiceCXNetwork(filename='SimpleNetwork.cx')
print(niceCx.getSummary())

## NDEx Networks

In [ ]:
niceCx = NiceCXNetwork(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')
print(niceCx.getSummary())


## NetworkX Networks

Networks can be manipulated using NetworkX facilities and then used to create a NiceCx network. Here we create a NetworkX network from scratch:

In [ ]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_node('GHI')
G.add_node('JKL')
G.add_node('MNO')
G.add_node('PQR')
G.add_node('XYZ')
G.add_edges_from([('ABC','DEF'), ('DEF', 'GHI'),('GHI', 'JKL'), 
                  ('DEF', 'JKL'), ('JKL', 'MNO'), ('DEF', 'MNO'),
                 ('MNO', 'XYZ'), ('DEF', 'PQR')])

We now use NetworkX to find the shortest path between nodes ABC and MNO and then create a subgraph from that path.

In [ ]:
short_path = nx.shortest_path(G,source='ABC',target="MNO")

path_subgraph = G.subgraph(short_path)

Finally, NiceCx objects are created from both *G* and *path_subgraph*.

In [ ]:
niceCx_full = NiceCXNetwork(networkx_G=G)
niceCx_full.setName('Created from NetworkX (full)')

niceCx_short = NiceCXNetwork(networkx_G=path_subgraph)
niceCx_short.setName('Created from NetworkX (shortest path)')
                       
print(niceCx_full.getSummary())
print(G.edges())
print('')

print(niceCx_short.getSummary())
print(H.edges())

## Pandas DataFrames

### 2 Column DataFrames with No Headers:

In [ ]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

niceCx = NiceCXNetwork(pandas_df=df)

print(niceCx.getSummary())

### 3 Column DataFrames with No Headers:

In [ ]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

niceCx = NiceCXNetwork(pandas_df=df)

print(niceCx.getSummary())

### Using 3+ Column DataFrames with Headers to Specify Attribute Columns

In [ ]:
df = pd.DataFrame.from_items([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

niceCx = NiceCXNetwork()

niceCx.create_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

print(niceCx.getSummary())